# IBM Developer Skills Network

In [1]:
import requests  # library to handle request
import pandas as pd
import numpy as np
import random  # library for random number generation

from geopy.geocoders import Nominatim  # convert an address into latitude and longitude

from IPython.display import Image     # library for displaying images
from IPython.core.display import HTML

from pandas.io.json import json_normalize  # transform json file into pandas dataframe

import folium  # map plotting library

## Define foursquare credentials and version

In [8]:
path = 'G:\Google Drive\FoursquareToken.csv'
df_token = pd.read_csv(path)

In [4]:
CLIENT_ID = df_token.iloc[0][1]
CLIENT_SECRET = df_token.iloc[1][1]
ACCESS_TOKEN = df_token.iloc[2][1]
VERSION = '20210526'
LIMIT = 30

In [9]:
address = '102 North End Ave, New York, NY'

geolocator = Nominatim(user_agent='foursquare_agent')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

40.7149555 -74.0153365


# Search for a specific venue category

In [10]:
search_query = 'Italian'
radius = 500
print(search_query + '...OK!')

Italian...OK!


## Define the corresponding URL

In [12]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)

## Send the get request and examine the results

In [13]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '60add71a7cb5d641c7638d42'},
 'notifications': [{'type': 'notificationTray', 'item': {'unreadCount': 0}}],
 'response': {'venues': [{'id': '4fa862b3e4b0ebff2f749f06',
    'name': "Harry's Italian Pizza Bar",
    'location': {'address': '225 Murray St',
     'lat': 40.71521779064671,
     'lng': -74.01473940209351,
     'labeledLatLngs': [{'label': 'display',
       'lat': 40.71521779064671,
       'lng': -74.01473940209351},
      {'label': 'entrance', 'lat': 40.715361, 'lng': -74.014975}],
     'distance': 58,
     'postalCode': '10282',
     'cc': 'US',
     'city': 'New York',
     'state': 'NY',
     'country': 'United States',
     'formattedAddress': ['225 Murray St',
      'New York, NY 10282',
      'United States']},
    'categories': [{'id': '4bf58dd8d48988d1ca941735',
      'name': 'Pizza Place',
      'pluralName': 'Pizza Places',
      'shortName': 'Pizza',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/pizza_',
  

## Transform json to dataframe

In [17]:
venues = results['response']['venues']  # get only relevant part
dataframe = pd.json_normalize(venues)
dataframe.head()

,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress
0,4fa862b3e4b0ebff2f749f06,Harry's Italian Pizza Bar,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",v-1622005530,False,225 Murray St,40.715218,-74.014739,"[{'label': 'display', 'lat': 40.71521779064671...",58,10282,US,New York,NY,United States,"[225 Murray St, New York, NY 10282, United Sta..."
1,4f3232e219836c91c7bfde94,Conca Cucina Italian Restaurant,"[{'id': '4d4b7105d754a06374d81259', 'name': 'F...",v-1622005530,False,63 W Broadway,40.714484,-74.009806,"[{'label': 'display', 'lat': 40.71448400000000...",469,10007,US,New York,NY,United States,"[63 W Broadway, New York, NY 10007, United Sta..."


## Define information of interest and filter dataframe

In [20]:
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]
dataframe_filtered.head()

,name,categories,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,id
0,Harry's Italian Pizza Bar,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",225 Murray St,40.715218,-74.014739,"[{'label': 'display', 'lat': 40.71521779064671...",58,10282,US,New York,NY,United States,"[225 Murray St, New York, NY 10282, United Sta...",4fa862b3e4b0ebff2f749f06
1,Conca Cucina Italian Restaurant,"[{'id': '4d4b7105d754a06374d81259', 'name': 'F...",63 W Broadway,40.714484,-74.009806,"[{'label': 'display', 'lat': 40.71448400000000...",469,10007,US,New York,NY,United States,"[63 W Broadway, New York, NY 10007, United Sta...",4f3232e219836c91c7bfde94


In [22]:
# Function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
    
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# Filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# Clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,id
0,Harry's Italian Pizza Bar,Pizza Place,225 Murray St,40.715218,-74.014739,"[{'label': 'display', 'lat': 40.71521779064671...",58,10282,US,New York,NY,United States,"[225 Murray St, New York, NY 10282, United Sta...",4fa862b3e4b0ebff2f749f06
1,Conca Cucina Italian Restaurant,Food,63 W Broadway,40.714484,-74.009806,"[{'label': 'display', 'lat': 40.71448400000000...",469,10007,US,New York,NY,United States,"[63 W Broadway, New York, NY 10007, United Sta...",4f3232e219836c91c7bfde94


## Visualize the Italian restaurant that are nearby

In [23]:
dataframe_filtered.name

0          Harry's Italian Pizza Bar
1    Conca Cucina Italian Restaurant
Name: name, dtype: object

In [25]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13)  # map around Conrad hotel

# Add a red circle mark to represent the hotel
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Conrad Hotel',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill=True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# Display map
venues_map

# Explore a Given Venue

## Explore the closest Italian restaurant

In [27]:
venue_id = '4fa862b3e4b0ebff2f749f06' # ID of Harry's Italian Pizza Bar
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&oauth_token={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET,ACCESS_TOKEN, VERSION)

## Get the result json file

In [28]:
result = requests.get(url).json()
print(result['response']['venue'].keys())
result['response']['venue']

dict_keys(['id', 'name', 'contact', 'location', 'canonicalUrl', 'categories', 'verified', 'stats', 'url', 'price', 'hasMenu', 'likes', 'like', 'dislike', 'ok', 'rating', 'ratingColor', 'ratingSignals', 'menu', 'allowMenuUrlEdit', 'beenHere', 'specials', 'photos', 'reasons', 'hereNow', 'createdAt', 'tips', 'shortUrl', 'timeZone', 'listed', 'hours', 'popular', 'seasonalHours', 'defaultHours', 'pageUpdates', 'inbox', 'attributes', 'bestPhoto', 'colors'])


{'id': '4fa862b3e4b0ebff2f749f06',
 'name': "Harry's Italian Pizza Bar",
 'contact': {'phone': '+12126081007', 'formattedPhone': '+1 212-608-1007'},
 'location': {'address': '225 Murray St',
  'lat': 40.71521779064671,
  'lng': -74.01473940209351,
  'labeledLatLngs': [{'label': 'display',
    'lat': 40.71521779064671,
    'lng': -74.01473940209351},
   {'label': 'entrance', 'lat': 40.715361, 'lng': -74.014975}],
  'postalCode': '10282',
  'cc': 'US',
  'city': 'New York',
  'state': 'NY',
  'country': 'United States',
  'formattedAddress': ['225 Murray St',
   'New York, NY 10282',
   'United States']},
 'canonicalUrl': 'https://foursquare.com/v/harrys-italian-pizza-bar/4fa862b3e4b0ebff2f749f06',
 'categories': [{'id': '4bf58dd8d48988d1ca941735',
   'name': 'Pizza Place',
   'pluralName': 'Pizza Places',
   'shortName': 'Pizza',
   'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/pizza_',
    'suffix': '.png'},
   'primary': True},
  {'id': '4bf58dd8d48988d110941735',
  

## Get venue overall rating

In [29]:
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet!')

6.9


In [31]:
venue_id = '4f3232e219836c91c7bfde94' # ID of Conca Cucina Italian Restaurant
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&oauth_token={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET,ACCESS_TOKEN, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet!')

This venue has not been rated yet!


In [35]:
venue_id = '3fd66200f964a520f4e41ee3' # ID of Ecco
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&oauth_token={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET,ACCESS_TOKEN, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

This venue has not been rated yet.


## Get the number of tips

In [36]:
result['response']['venue']['tips']['count']

19

## Get the venue's tips

In [37]:
limit = 19 # set limit to be greater than or equal to the total number of tips
url = 'https://api.foursquare.com/v2/venues/{}/tips?client_id={}&client_secret={}&oauth_token={}&v={}&limit={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET,ACCESS_TOKEN, VERSION, limit)

results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '60ade17c7964105656b6e02a'},
 'notifications': [{'type': 'notificationTray', 'item': {'unreadCount': 0}}],
 'response': {'tips': {'count': 18,
   'items': [{'id': '5ab1cb46c9a517174651d3fe',
     'createdAt': 1521601350,
     'text': 'A+ Italian food! Trust me on this: my mom’s side of the family is 100% Italian. I was born and bred to know good pasta when I see it, and Ecco is one of my all-time NYC favorites',
     'type': 'user',
     'canonicalUrl': 'https://foursquare.com/item/5ab1cb46c9a517174651d3fe',
     'likes': {'count': 0, 'groups': []},
     'like': False,
     'logView': True,
     'agreeCount': 4,
     'disagreeCount': 0,
     'todo': {'count': 0},
     'user': {'id': '484542633',
      'firstName': 'Nick',
      'lastName': 'El-Tawil',
      'gender': 'male',
      'address': '',
      'city': '',
      'state': '',
      'photo': {'prefix': 'https://fastly.4sqi.net/img/user/',
       'suffix': '/484542633_64VpAzxd_1j4zzJ30WhPZsmHMDuV

## Get tips and list of associated features

In [38]:
tips = results['response']['tips']['items']
tip = results['response']['tips']['items'][0]
tip.keys()

dict_keys(['id', 'createdAt', 'text', 'type', 'canonicalUrl', 'likes', 'like', 'logView', 'agreeCount', 'disagreeCount', 'todo', 'user', 'authorInteractionType'])

## Format column width and display all tips

In [41]:
pd.set_option('display.max_colwidth', None)

tips_df = pd.json_normalize(tips)

# columns to keep
filtered_columns = ['text', 'agreeCount', 'disagreeCount', 'id', 'user.firstName', 'user.lastName', 'user.id']
tips_filtered = tips_df.loc[:, filtered_columns]

# display tips
tips_filtered.reindex()

,text,agreeCount,disagreeCount,id,user.firstName,user.lastName,user.id
0,"A+ Italian food! Trust me on this: my mom’s side of the family is 100% Italian. I was born and bred to know good pasta when I see it, and Ecco is one of my all-time NYC favorites",4,0,5ab1cb46c9a517174651d3fe,Nick,El-Tawil,484542633


# Search a FourSquare User

In [43]:
idnumber = '484542633' # user ID with most agree counts and complete profile

url = 'https://api.foursquare.com/v2/users/{}/?client_id={}&client_secret={}&oauth_token={}&v={}'.format(idnumber,CLIENT_ID, CLIENT_SECRET, ACCESS_TOKEN,VERSION) # define URL

results = requests.get(url).json()

user_data = results['response']['user']['photos']['items']

# result
pd.set_option('display.max_colwidth', None)

users_df = pd.json_normalize(user_data)

# For later display of user image
filtered_columns = ['id', 'prefix', 'suffix', 'width', 'height']
tips_filtered = users_df.loc[:, filtered_columns]

tips_filtered

,id,prefix,suffix,width,height
0,5e41a77864fa070008131752,https://fastly.4sqi.net/img/general/,/484542633_ELnUC1di2LwJTjPi04McysQZNqJHSCCSxS3i_GKGTEY.jpg,1440,1440


In [44]:
g=tips_df.loc[tips_df['user.id'] == '484542633']
print('First Name: ' + tips_df['user.firstName'])
print('Last Name: ' + tips_df['user.lastName'])

0    First Name: Nick
Name: user.firstName, dtype: object
0    Last Name: El-Tawil
Name: user.lastName, dtype: object


## Retrieve the User's profile image

In [45]:
# 1. grab prefix of photo 
# 2. grab suffix of photo
# 3. concatenate them using the image size  
Image(url='https://fastly.4sqi.net/img/general/540x920/484542633_ELnUC1di2LwJTjPi04McysQZNqJHSCCSxS3i_GKGTEY.jpg')

## Get user's tips

In [48]:
# define tips URL
user_id='484542633'
url = 'https://api.foursquare.com/v2/users/{}/tips?client_id={}&client_secret={}&oauth_token={}&v={}&limit={}'.format(user_id, CLIENT_ID, CLIENT_SECRET,ACCESS_TOKEN,VERSION, limit)

# send GET request and get user's tips
results = requests.get(url).json()
tips = results['response']['tips']['items']

# format column width
pd.set_option('display.max_colwidth', None)

tips_df = pd.json_normalize(tips)

# filter columns
filtered_columns = ['text', 'agreeCount', 'disagreeCount', 'id']
tips_filtered = tips_df.loc[:, filtered_columns]

# display user's tips
tips_filtered

,text,agreeCount,disagreeCount,id
0,"High quality falafel that doesn’t come from a cart on the side of the road?! Yes, it exists, and it exists at Taim. Highly recommended - even if you’re new to falafel.",6,0,5ab2771a7dc9e17930670085
1,"If you are in the West Village and you choose to go to a nationwide chain coffee shop instead of 11th Street Cafe, then you are everything that is wrong with the world. This place is AWESOME!!!",3,0,5ab1daf0da5e5617d1da3c7d
2,A+! The New Whitney is MUCH more accessible and easier to navigate than the city’s other famous museums like the Met. I’m a big fan. If you’re on a budget: there’s free admission on Friday nights,5,0,5ab1d649464d655c24fc144f
3,"I had the wings and they blew my mind. Plus, Leah (the owner) is pretty awesome, so you should definitely check it out!",1,0,5ab1d0a5c666662673a11799
4,Everything here is fantastic! I’ve ordered half the menu and never been disappointed. My favorite is the spaghetti & meatballs (only served on Sundays). The spicy sauce takes it to the next level!,6,0,5ab1cbe467f62b57ee6beaa9
5,"THE spot for breakfast in the West Village. I recommend ordering the healthy treat. If you don’t like it, then you have no soul.",5,0,5ab1af48f79faa4bc867de78
6,Get the vegan peanut butter bar. It’s like sex in your mouth.,1,0,5ab1ad2847f8767a8ca88fc1
7,"My go-to for coffee and a breakfast sandwich in the mornings. That actually undersells this place - EVERYTHING is good. Lunch, pasta, desserts, wine. I’ve never been disappointed here.",2,0,5ab1acbba6ec980645fd7512
8,This is the real deal. Stuff like this is why I moved to NYC. I bet half of the guys who play here could have gone pro. Do NOT bring your cousin Jimmy (who never played before) unless you like losing!,1,0,5ab1a316a8eb606122a3b755
9,The High Line is pretty cool. The only thing not to like is that it’s so good that everyone comes here and it gets too crowded. Check it out on a weekday morning or at night to dodge the crowds.,10,0,5ab19f6849281477a8be2dab


## Get the venue for the tip with the greatest number of agree counts

In [49]:
tip_id = '5ab5575d73fe2516ad8f363b' # tip id

# define URL
url = 'https://api.foursquare.com/v2/users/{}/tips?client_id={}&client_secret={}&oauth_token={}&v={}'.format(idnumber, CLIENT_ID, CLIENT_SECRET,ACCESS_TOKEN, VERSION) # define URL

# send GET Request and examine results
result = requests.get(url).json()
print(result['response']['tips']['items'][0]['venue']['name'])
print(result['response']['tips']['items'][0]['venue']['location'])

Taïm
{'address': '222 Waverly Pl', 'crossStreet': 'btwn Perry & W 11th St', 'lat': 40.735970102491926, 'lng': -74.00194019079207, 'labeledLatLngs': [{'label': 'display', 'lat': 40.735970102491926, 'lng': -74.00194019079207}, {'label': 'entrance', 'lat': 40.735887, 'lng': -74.001873}], 'postalCode': '10014', 'cc': 'US', 'neighborhood': 'West Village', 'city': 'New York', 'state': 'NY', 'country': 'United States', 'formattedAddress': ['222 Waverly Pl (btwn Perry & W 11th St)', 'New York, NY 10014', 'United States']}


# Explore a location

In [50]:
latitude = 40.715337
longitude = -74.008848

In [51]:
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)

In [52]:
results = requests.get(url).json()
'There are {} around Ecco restaurant.'.format(len(results['response']['groups'][0]['items']))

'There are 30 around Ecco restaurant.'

## Get relevant part of JSON

In [53]:
items = results['response']['groups'][0]['items']
items[0]

{'reasons': {'count': 0,
  'items': [{'summary': 'This spot is popular',
    'type': 'general',
    'reasonName': 'globalInteractionReason'}]},
 'venue': {'id': '4af5d65ff964a52091fd21e3',
  'name': 'Korin',
  'location': {'address': '57 Warren St',
   'crossStreet': 'Church St',
   'lat': 40.71482437714839,
   'lng': -74.00940425461492,
   'labeledLatLngs': [{'label': 'display',
     'lat': 40.71482437714839,
     'lng': -74.00940425461492},
    {'label': 'entrance', 'lat': 40.714727, 'lng': -74.009399}],
   'distance': 73,
   'postalCode': '10007',
   'cc': 'US',
   'neighborhood': 'Tribeca',
   'city': 'New York',
   'state': 'NY',
   'country': 'United States',
   'formattedAddress': ['57 Warren St (Church St)',
    'New York, NY 10007',
    'United States']},
  'categories': [{'id': '4bf58dd8d48988d1f8941735',
    'name': 'Furniture / Home Store',
    'pluralName': 'Furniture / Home Stores',
    'shortName': 'Furniture / Home',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/cate

## Process JSON and turn it to dataframe

In [55]:
dataframe = pd.json_normalize(items) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories'] + [col for col in dataframe.columns if col.startswith('venue.location.')] + ['venue.id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# filter the category for each row
dataframe_filtered['venue.categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean columns
dataframe_filtered.columns = [col.split('.')[-1] for col in dataframe_filtered.columns]

dataframe_filtered.head(5)

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,neighborhood,city,state,country,formattedAddress,id
0,Korin,Furniture / Home Store,57 Warren St,Church St,40.714824,-74.009404,"[{'label': 'display', 'lat': 40.71482437714839, 'lng': -74.00940425461492}, {'label': 'entrance', 'lat': 40.714727, 'lng': -74.009399}]",73,10007,US,Tribeca,New York,NY,United States,"[57 Warren St (Church St), New York, NY 10007, United States]",4af5d65ff964a52091fd21e3
1,Los Tacos No. 1,Taco Place,136 Church St,NaN,40.714267,-74.008756,"[{'label': 'display', 'lat': 40.714267, 'lng': -74.008756}]",119,10007,US,NaN,New York,NY,United States,"[136 Church St, New York, NY 10007, United States]",5d5f24ec09484500079aee00
2,Juice Press,Vegetarian / Vegan Restaurant,83 Murray St,btwn Greenwich St & W Broadway,40.714788,-74.011132,"[{'label': 'display', 'lat': 40.71478769908051, 'lng': -74.0111317502157}]",202,10007,US,NaN,New York,NY,United States,"[83 Murray St (btwn Greenwich St & W Broadway), New York, NY 10007, United States]",54148bc6498ea7bb8c05b70a
3,Takahachi Bakery,Bakery,25 Murray St,at Church St,40.713653,-74.008804,"[{'label': 'display', 'lat': 40.713652845301894, 'lng': -74.0088038953017}, {'label': 'entrance', 'lat': 40.713716, 'lng': -74.008443}]",187,10007,US,NaN,New York,NY,United States,"[25 Murray St (at Church St), New York, NY 10007, United States]",4c154c9a77cea593c401d260
4,Heyday,Spa,92 Reade St,NaN,40.715726,-74.007767,"[{'label': 'display', 'lat': 40.715726, 'lng': -74.007767}, {'label': 'entrance', 'lat': 40.715654, 'lng': -74.00782}]",100,10013,US,NaN,New York,NY,United States,"[92 Reade St, New York, NY 10013, United States]",57ad129c498e05b086594d72


## Visualize the map

In [56]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=15) # generate map centred around Ecco


# add Ecco as a red circle mark
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    popup='Ecco',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(venues_map)


# add popular spots to the map as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        fill=True,
        color='blue',
        fill_color='blue',
        fill_opacity=0.6
        ).add_to(venues_map)

# display map
venues_map

# Explore trending value

In [57]:
# define URL
url = 'https://api.foursquare.com/v2/venues/trending?client_id={}&client_secret={}&ll={},{}&v={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION)

# send GET request and get trending venues
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '60adf02c2a3aee6c4880fc76'},
 'response': {'venues': []}}

## Check if any venues are trending at this time

In [58]:
if len(results['response']['venues']) == 0:
    trending_venues_df = 'No trending venues are available at the moment!'
    
else:
    trending_venues = results['response']['venues']
    trending_venues_df = json_normalize(trending_venues)

    # filter columns
    columns_filtered = ['name', 'categories'] + ['location.distance', 'location.city', 'location.postalCode', 'location.state', 'location.country', 'location.lat', 'location.lng']
    trending_venues_df = trending_venues_df.loc[:, columns_filtered]

    # filter the category for each row
    trending_venues_df['categories'] = trending_venues_df.apply(get_category_type, axis=1)

In [59]:
# display trending venues
trending_venues_df

'No trending venues are available at the moment!'

## Visualize trending venues

In [60]:
if len(results['response']['venues']) == 0:
    venues_map = 'Cannot generate visual as no trending venues are available at the moment!'

else:
    venues_map = folium.Map(location=[latitude, longitude], zoom_start=15) # generate map centred around Ecco


    # add Ecco as a red circle mark
    folium.CircleMarker(
        [latitude, longitude],
        radius=10,
        popup='Ecco',
        fill=True,
        color='red',
        fill_color='red',
        fill_opacity=0.6
    ).add_to(venues_map)


    # add the trending venues as blue circle markers
    for lat, lng, label in zip(trending_venues_df['location.lat'], trending_venues_df['location.lng'], trending_venues_df['name']):
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            poup=label,
            fill=True,
            color='blue',
            fill_color='blue',
            fill_opacity=0.6
        ).add_to(venues_map)

In [61]:
# display map
venues_map

'Cannot generate visual as no trending venues are available at the moment!'